## This compares to DoubleBracketIteration whenever possible

In [1]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo import symbols
from double_bracket_evolution_oracles import *
from group_commutator_iteration_transpiler import *
from numpy.linalg import norm
"""Test create evolution oracle"""

h_x = SymbolicHamiltonian( symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) 
                              + symbols.Y(1) * symbols.Y(2), nqubits = 3 )
d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits = 3 )
# Initialize with EvolutionOracleType hamiltonian_simulation so that d_0 is Delta
h_input = h_x + d_0



[Qibo 0.2.5|INFO|2024-02-28 08:38:48]: Using qibojit (numba) backend on /CPU:0


In [2]:
t_step = 0.0051#0.98#dbi.hyperopt_step()

#### 1. DoubleBracketIteration and group commutator

In [3]:
from double_bracket import *

dbi = DoubleBracketIteration(deepcopy(h_input.dense))
dbi.mode = DoubleBracketGeneratorType.group_commutator

[Qibo 0.2.5|WARNING|2024-02-28 08:38:48]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


DoubleBracketIteration only rotates the hamiltonian h, and currently doesn't give access to the unitary

Instead, we can wrap around the code executed in __call__

In [4]:
def wrapper_gc(self,step, d):
    return (
                self.h.exp(-step)
                @ self.backend.calculate_matrix_exp(-step, d)
                @ self.h.exp(step)
                @ self.backend.calculate_matrix_exp(step, d)
            )
unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),d_0.dense.matrix)

[Qibo 0.2.5|WARNING|2024-02-28 08:38:48]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [5]:
dbi(t_step, d = d_0.dense.matrix)


#### 2. Evolution oracle hamiltonian simulation


In [6]:
evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_hamiltonian_simulation ))

In [7]:
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)



In [8]:
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

In [9]:
norm(unitary_gc_from_oracles['forwards'].unitary() - unitary_gc_existing)

0.007068588046085704

In [10]:
norm(unitary_gc_from_oracles['backwards'].unitary() - unitary_gc_existing)

0.05706730540522153

We may improve the discrepancy by setting smaller eps

In [11]:
gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = 0.0000001
evolution_oracle_diagonal_target.eps_trottersuzuki = 0.00000001
gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = True
evolution_oracle_diagonal_target.please_be_verbose = False

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
norm(unitary_gc_from_oracles['forwards'].unitary() - unitary_gc_existing)

Calling circuit in Hamiltonian simulation mode for time t=-0.07141428428542851 and next running discretization adjustment to reach precision eps = 1e-07
3 5.703698185439878e-05
6 1.425710829003397e-05
12 3.5641435459211722e-06
24 8.910275418042565e-07
48 2.2275636394665562e-07
96 5.568905837658941e-08
Calling circuit in Hamiltonian simulation mode for time t=0.07141428428542851 and next running discretization adjustment to reach precision eps = 1e-07
3 5.703698185442657e-05
6 1.4257108290027298e-05
12 3.5641435458984836e-06
24 8.910275417724017e-07
48 2.2275636393673975e-07
96 5.568905839345736e-08
Calling circuit in Hamiltonian simulation mode for time t=-0.07141428428542851 and next running discretization adjustment to reach precision eps = 1e-07
3 5.703698185439878e-05
6 1.425710829003397e-05
12 3.5641435459211722e-06
24 8.910275418042565e-07
48 2.2275636394665562e-07
96 5.568905837658941e-08
Calling circuit in Hamiltonian simulation mode for time t=0.07141428428542851 and next runn

0.00706869515351788

In [12]:

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
norm(unitary_gc_from_oracles['backwards'].unitary() - unitary_gc_existing)

Calling circuit in Hamiltonian simulation mode for time t=-0.07141428428542851 and next running discretization adjustment to reach precision eps = 1e-07
3 5.703698185439878e-05
6 1.425710829003397e-05
12 3.5641435459211722e-06
24 8.910275418042565e-07
48 2.2275636394665562e-07
96 5.568905837658941e-08
Calling circuit in Hamiltonian simulation mode for time t=0.07141428428542851 and next running discretization adjustment to reach precision eps = 1e-07
3 5.703698185442657e-05
6 1.4257108290027298e-05
12 3.5641435458984836e-06
24 8.910275417724017e-07
48 2.2275636393673975e-07
96 5.568905839345736e-08
Calling circuit in Hamiltonian simulation mode for time t=-0.07141428428542851 and next running discretization adjustment to reach precision eps = 1e-07
3 5.703698185439878e-05
6 1.425710829003397e-05
12 3.5641435459211722e-06
24 8.910275418042565e-07
48 2.2275636394665562e-07
96 5.568905837658941e-08
Calling circuit in Hamiltonian simulation mode for time t=0.07141428428542851 and next runn

0.009965571871141131

#### 3. Evolution oracle numpy


In [13]:
evolution_oracle_numerical = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.numerical)

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_numerical ))

In [14]:
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.numerical)


In [15]:
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

Compared to the group commutator using Hamiltonian simulation there will be small deviations that arise from Trotter-Suzuki decomposition

In [16]:
norm(unitary_gc_from_oracles['backwards'] - unitary_gc_existing)

0.0040927048339044324

In [17]:
norm(unitary_gc_from_oracles['forwards'] - unitary_gc_existing)

0.057357576681930346

We may check by switching the group commutator flag that the difference comes from ordering and inversions

In [18]:
gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting

unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
norm(unitary_gc_from_oracles['forwards'] - unitary_gc_existing)

0.0

In [20]:
evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.numerical)
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_hamiltonian_simulation ))

In [25]:
assert evolution_oracle_hamiltonian_simulation.mode_evolution_oracle.value == 3

In [27]:
assert evolution_oracle_diagonal_target.mode_evolution_oracle.value == 2

In [21]:
used_circuit = gci(t_step, diagonal_association= evolution_oracle_diagonal_target)

AssertionError: 

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
gc_numpy = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.numerical))

In [ ]:
## Test more fancy functionalities
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = False
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(input_hamiltonian_evolution_oracle_hamiltonian_simulation ))
d_ev =  EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)

gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
query_list = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= d_ev )

norm(query_list['forwards'].unitary() -query_list['backwards'].unitary().conj().T)




In [ ]:
norm(query_list['forwards'] -query_list['backwards'].T.conj())

In [ ]:
#Test file entry
u = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step)).unitary()
u2 = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step)).unitary()
norm(u-u2.T.conj())

In [ ]:
d_0.mode_evolution_oracle = EvolutionOracleType.text_strings
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.text_strings
query_list = gci.group_commutator( np.sqrt(t_step*2),
             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0"))


query_list['forwards']